In [0]:
%sh pip install regex
pip install textblob
pip install elasticsearch

Requirement already satisfied: regex in /databricks/python3/lib/python3.7/site-packages (2020.11.13)
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already satisfied: textblob in /databricks/python3/lib/python3.7/site-packages (0.15.3)
Requirement already satisfied: nltk>=3.1 in /databricks/python3/lib/python3.7/site-packages (from textblob) (3.5)
Requirement already satisfied: click in /databricks/python3/lib/python3.7/site-packages (from nltk>=3.1->textblob) (7.1.2)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.7/site-packages (from nltk>=3.1->textblob) (4.51.0)
Requirement already satisfied: regex in /databricks/python3/lib/python3.7/site-packages (from nltk>=3.1->textblob) (2020.11.13)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.7/site-packages (from nltk>=3.1->textblob) (0.17.0)
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Collecting elasticsearch
 Downloading https://files.pythonhosted.org/packages/14/ba/f950bdd9164fb2bbbe5093700162234fbe61f446fe2300a8993761c132ca/elasticsearch-7.10.0-py2.py3-none-any.whl (321kB)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.7/site-packages (from elasticsearch) (2019.3.9)
Requirement already satisfied: urllib3<2,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from elasticsearch) (1.24.1)
Installing collected packages: elasticsearch
Successfully installed elasticsearch-7.10.0
You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [0]:
from kafka import KafkaConsumer
from json import loads
import json
from textblob import TextBlob
import re
from elasticsearch import Elasticsearch
from pyspark.sql import SQLContext

consumer = KafkaConsumer(
    'numtest',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

def getHashTag(text):
    if "trump" in text.lower():
        return "#trump"
    else:
        return "#corona"

def clean_tweet(tweet): 
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 


def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_tweet(tweet)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'
      
def getSentiment(rdd):
  test = rdd.collect()
  print(test[0])
  d = {"sentiment": test[0][0], "hashtag": test[0][1], "tweet":test[0][2]}
  esConn.index(index="hash_tags_sentiment_analysis", doc_type="tweet-sentiment-analysis", body=d)
  

esConn = Elasticsearch([{'host': '', 'port': 9200}])
for message in consumer:
    #message = [getHashTag(message.value),get_tweet_sentiment(message.value), message.value]
    df = sc.parallelize([message.value])
    rdd = df.map(lambda x: (get_tweet_sentiment(x), getHashTag(message.value), x))
    #new_rdd = rdd.map(json.dumps).map(lambda x: ('key', x))
    getSentiment(rdd)